# data

> Prepare data for training

In [ ]:
#| default_exp data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import pickle
from pathlib import Path
from copy import copy

import numpy as np
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
import pandas as pd

from prody import AtomGroup, parsePDB

import torch

from torch_geometric.data import Data, InMemoryDataset

from fastcore.script import *

In [ ]:
#| export

def get_ca_coords_from_parsed_prody(parsed_structure_prody: AtomGroup):
    """Extract 3D coordinates of alpha carbons from a parsed ProDy structure."""
    coords = {}
    for chain in parsed_structure_prody.getHierView():
        chain_id = chain.getChid()
        chain_ca = chain.select("name CA")
        chain_coords = chain_ca.getCoords()
        assert len(chain_coords) == len(chain_ca.getSequence())
        coords[chain_id] = chain_coords

    return coords


@call_parse
def create_raw_dataset(
    structures_dir: Param("Directory containing structures in PDB format", str),
    esm_embeddings_dir: Param("Directory containing ESM embeddings", str),
    labels_path: Param("Path for target labels in text format", str),
    output_dir: Param("Output directory for the dataset", str),
):
    structures_dir = Path(structures_dir)
    esm_embeddings_dir = Path(esm_embeddings_dir)
    labels_path = Path(labels_path)
    output_dir = Path(output_dir)
    output_dir_raw = output_dir / "raw"
    output_dir_raw.mkdir(parents=True, exist_ok=True)

    sorted_structure_paths = sorted(structures_dir.glob("*.pdb"))

    # Load target labels into a pandas dataframe
    Y = pd.read_csv(
        labels_path,
        sep=" ",
        header=None,
        names=["Structure", "Value"],
        index_col="Structure",
    ).squeeze()

    raw_dataset = []
    for idx, structure_path in enumerate(sorted_structure_paths):
        raw_data = {"idx": torch.tensor([idx])}

        name = structure_path.name.removesuffix(".pdb")
        raw_data["name"] = name

        # Get 3D Ca coordinates
        parsed_structure_prody = parsePDB(str(structure_path))
        coords = get_ca_coords_from_parsed_prody(parsed_structure_prody)
        raw_data["pos"] = torch.cat(
            [
                torch.from_numpy(chain_coords).to(torch.float32)
                for chain_coords in coords.values()
            ]
        )

        # Get ESM embeddings
        with open(esm_embeddings_dir / f"{name}.pkl", "rb") as f:
            esm_embeddings = pickle.load(f)
        raw_data["esm_embeddings"] = torch.cat(
            [esm_embeddings[chain_id][0][0][1:-1] for chain_id in coords]
        )
        assert len(raw_data["esm_embeddings"]) == len(raw_data["pos"])

        raw_data["y"] = torch.tensor([[Y[name]]]).to(torch.float32)

        raw_dataset.append(raw_data)

    torch.save(raw_dataset, output_dir_raw / "data.pt")

    return raw_dataset


def make_train_val_split_clustering_by_rmsd(rmsds_matrix, cutoff, train_frac=0.8):
    Z = linkage(squareform(rmsds_matrix, checks=False), method="complete")
    labels = fcluster(Z, cutoff, criterion="distance")
    unique, inverse = np.unique(labels, return_inverse=True)
    unique_shuffled = unique.copy()
    np.random.shuffle(unique_shuffled)
    train_labels, val_labels = np.split(
        unique_shuffled, [int(train_frac * len(unique))]
    )

    train_idxs = []
    val_idxs = []
    for i, label in enumerate(inverse):
        if np.isin(label, train_labels):
            train_idxs.append(i)
        else:
            val_idxs.append(i)

    return train_idxs, val_idxs


class JClinicDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return "data.pt"

    @property
    def processed_file_names(self):
        return "data.pt"

    def download(self):
        pass

    def process(self):
        # Read data into a `Data` list
        data_list = [
            Data(**raw_dict)
            for raw_dict in torch.load(Path(self.raw_dir) / self.raw_file_names)
        ]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
show_doc(get_ca_coords_from_parsed_prody)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()